#Yahoo Finance Library

In [ ]:
!pip install msgpack
!pip install yfinance
!pip install pandas-datareader

In [ ]:
# yahoo_finance ->
#fix_yahoo_finance ->
#yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import datetime
from pandas_datareader import data as pdr
pd.core.common.is_list_like = pd.api.types.is_list_like

In [ ]:
yf.pdr_override()

In [ ]:
samsung_code = '005930.KS' # code / Ticker,  KS를 붙여 한국꺼임을 알려줌
start_date = datetime.date(2017, 7, 1)
end_date = datetime.date(2021, 7, 14)
#pdr.get_data_yahoo(samsung_code, start_date, end_date) # 종목, 시작일자, 끝나는 일자
# Adjusted Close price(수정종가)


In [ ]:
pdr.get_data_yahoo('AAPL',start_date,end_date)

In [ ]:
aapl = yf.Ticker('AAPL')

In [ ]:
aapl.info

In [ ]:
# get history market data
hist = aapl.history(period='max')
hist

In [ ]:
# show actions (dividends, splits, etc.)
aapl.actions

In [ ]:
# dividends
aapl.dividends

In [ ]:
# financial  statments
# aapl.financials
#aapl.quarterly_financials

In [ ]:
aapl.major_holders

In [ ]:
# 기관투자자만 보기
aapl.institutional_holders

In [ ]:
aapl.sustainability

In [ ]:
aapl.recommendations

In [ ]:
aapl.calendar

In [ ]:
# ISIN = Internatinal Securities Identification Number
aapl.isin

In [ ]:
# 옵션 만기일 show options expirations
aapl.options

In [ ]:
# 만기일 별로 상세한 정보 
aapl.option_chain('2021-07-16')[0]

# 추세투자법

|레벨|스타일|기대CAGR|매수전략|매도전략|
|:--:|:--:|:--:|:--:|:--:|
|초급|자산배분+모멘텀|코스피 + 2~3%|3개월 이동평균이 상승 반전하면 코스피 지수 매수|3개월 이동평균이 하락 반전하면 코스피 지수 매도한 후 전액 현금 보유|

In [ ]:
'''
기대CAGR : 코스피 + 2~3%
매수전략 : 3개월 이동평균이 상승 반전하면 콯스피 지수 매수
매도전략 : 3개월 이동평균이 하락 반전하면 코스피 지수 매도한 후 전액 현금 보유
'''

###Import libraries


In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

### Get KOSPI 200 market data(Daily)

코스피 지수를 네이버 금융에서 크롤링합니다. 크롬의 검사(Inspection)기능을 사용해 개발자 도구를 열고 크롤링해오려는 데이터 표의 source URL을 먼저 변수로 지정합니다.
크롤링해온 일별 데이터를 `kospi`에 DataFrame으로 담습니다.

In [ ]:
kospi_200 = pd.DataFrame()
kospi_url = 'https://finance.naver.com/item/sise_day.nhn?code=069500' 
for page in tqdm(range(1, 100)):
    
    pg_url = '{url}&page={page}'.format(url=kospi_url, page=page)
    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    kospi_200 = kospi_200.append(pd.read_html(pg_url)[0], ignore_index=True)

kospi_200 = kospi_200.dropna()
kospi_200.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
kospi_200.date = pd.to_datetime(kospi_200.date)




In [ ]:
kospi_200 = kospi_200.sort_values(by='date')

In [ ]:
kospi_200.head(5)

`rolling`이라는 객체 반환 후 90일 데이터를 셀렉트한 후 `mean`으로 평균을 구합니다


In [ ]:
mavg_90 = kospi_200['close'].rolling(window=90).mean() 
mavg_90.tail()


만든 90일 이동평균을 새로운 column으로 만들어 넣습니다.

In [ ]:
kospi_200['MAVG_90'] = mavg_90
kospi_200 = kospi_200.dropna()

In [ ]:
print(mavg_90.shape)

In [ ]:
kospi_200

# Visualization

이동평균선을 그래프로 나타냅니다.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(18, 4))

plt.plot(kospi_200.date, kospi_200.close, label='close')
plt.plot(kospi_200.date, kospi_200.MAVG_90, label='MAVG_90')

plt.legend(loc='best')
plt.xlabel('Year', fontsize=18)
plt.ylabel('Index', fontsize=18 )
plt.grid()


plt.show()

# Backtesting

In [ ]:
def backtest(beginning_asset = 100000): # 최초 금액은 $100,000로 시작
    asset = [beginning_asset] # 최종적으로 반환할 월별 자산 데이터가 담길 리스트를 선언 후, 최초 금액을 담아둡니다
    kospi_shares = 0 # 첫 시점에서 KODEX 주식 수는 0주입니다.
    for i in tqdm(range(len(kospi_200)-1)): # 월별 데이터의 개수만큼 반복
        # 이전 시점(날) 코스피 이평선 대비 현 시점 코스피 이평선이 상승했을 경우,
        if kospi_200.iloc[i-1].MAVG_90 < kospi_200.iloc[i].MAVG_90:
            kospi_shares = beginning_asset / kospi_200.iloc[i].close # 해당 시점 종가로 매수가능한 코스피 주식 수를 계산
            kospi_ending_value = kospi_shares * kospi_200.iloc[i+1].close # 다음 시점 종가와 주식수를 곱해 기말 자산액을 구함

            asset.append(kospi_ending_value) # 총 자산가치를 asset 리스트에 담습니다
            beginning_asset = kospi_ending_value # i+1번째 회차의 총 자산가치가 곧 i+2번째 회차의 초기자산이 되기 때문에 재설정


        # 이전 시점(날) 코스피 이평선 대비 현 시점 코스피 이평선이 하락했을 경우,
        elif kospi_200.iloc[i-1].MAVG_90 > kospi_200.iloc[i].MAVG_90:
            kospi_shares = 0 # 코스피 주식은 전량매도하여 주식수 0으로 설정
            kospi_ending_value = beginning_asset

            asset.append(kospi_ending_value)
            beginning_asset = kospi_ending_value            
    

    # asset을 반환합니다.
    return asset

In [ ]:
result = backtest()


In [ ]:
len(result)

백테스트 결과와 코스피 지수를 DataFrame으로 변환합니다.

In [ ]:
adj_kospi = kospi_200.close / kospi_200.iloc[0].close * 100000
back_test = {
    'date' : kospi_200.date,
    'backtest' : result,
    'kospi' : adj_kospi
}
back_test = pd.DataFrame(back_test, columns= ['date', 'backtest', 'kospi'])
back_test.head()


In [ ]:
back_test

#Visualization

In [ ]:
# 백테스트 결과를 그래프로 나타냅니다.
plt.figure(figsize=(18, 4))
plt.plot(kospi_200.date, result)
plt.grid()
plt.show()

결과 DataFrame을 그래프로 시각화합니다.

In [ ]:
plt.figure(figsize=(18, 4))
plt.plot(back_test.date, back_test.backtest, label='Backtest', color='green', linestyle='dashed')
plt.plot(back_test.date, back_test.kospi, label='KOSPI', color='red')

plt.title('Backtesting Portfolio & Benchmark', fontsize=25)
plt.xlabel("Year", fontsize=18)
plt.ylabel("Asset", fontsize=18)
plt.grid()
plt.legend(loc='best')
plt.show()

#Compute CAGR

백테스트 결과 값과 코스피의 연간복리수익률을 계산합니다.

In [ ]:
start, end = back_test.backtest.iloc[0], back_test.backtest.iloc[-1]
cagr = ((end/start)**(1/len(set(back_test.date.dt.year)))-1) * 100
print('%.2f'%cagr, '%')

In [ ]:
start, end = back_test.kospi.iloc[0], back_test.kospi.iloc[-1]
cagr = ((end/start)**(1/len(set(back_test.date.dt.year))) -1 )*100
print('%.2f'%cagr, '%')

## Compute Statistics

`numpy`모듈을 사용해 통계치를 계산합니다.

In [ ]:
# standard deviation of portfolio in percentage
(np.std(back_test) / np.mean(back_test)) * 100

`diff`를 사용해 매월 자산액의 차이를 계산한 후 전월대비 수익률을 계산


In [ ]:
back_test['Diff'] = back_test['backtest'].diff()
back_test.head()

In [ ]:
back_test['KOSPI_Diff'] = back_test["kospi"].diff()
back_test.head()

In [ ]:
ror_list = []
for i in range(len(back_test)):
    ror_list.append(back_test['Diff'].iloc[i] / back_test['backtest'].iloc[i-1])
back_test['RoR'] = ror_list

In [ ]:
ror_list = []
for i in range(len(back_test)):
    ror_list.append(back_test['KOSPI_Diff'].iloc[i] / back_test['kospi'].iloc[i-1])
back_test['KOSPI_RoR'] = ror_list

In [ ]:
back_test['RoR'].iloc[0] = 0
back_test['RoR'].iloc[1] = 0
back_test['KOSPI_RoR'].iloc[0] = 0
back_test['KOSPI_RoR'].iloc[1] = 0

In [ ]:
back_test.head()

In [ ]:
# for i in tqdm(range(len(back_test)-1)):
#     back_test["RoR"] = (back_test["backtest"].diff() / back_test.iloc[i]['backtest'])*100
#     back_test['KOSPI_RoR'] = (back_test['kospi'].diff() / back_test.iloc[i]['kospi'])*100
# back_test.fillna(0, inplace=True)
# back_test.head()

수익률의 표준편차를 계산합니다.


In [ ]:
# absolute standard deviation
np.std(back_test) # n for all
back_test.std() # n-1 for sample

In [ ]:
# relative standard deviation
(np.std(back_test)/ np.mean(back_test)) * 100 # n for all
(back_test.std() / back_test.mean()) *100 # n-1 for sample

수익률의 표준편차가 최대치가 되는 시점을 찾습니다




In [ ]:
np.datetime_as_string(back_test[back_test['RoR'] == max(back_test['RoR'])].date.values[0], unit='D'), max(back_test['RoR'])

수익률의 표준편차가 최소치가 되는 시점 찾습니다


In [ ]:
np.datetime_as_string(back_test[back_test['RoR'] == min(back_test['RoR'])].date.values[0], unit='D'), min(back_test['RoR'])

백테스트 수익률 분포를 히스토그램으로 나타냅니다.


In [ ]:
plt.hist(back_test['RoR'], bins=100)
plt.show()

KOSPI 수익률 분포를 히스토그램으로 나타냅니다


In [ ]:
plt.hist(back_test['KOSPI_RoR'], bins=100)
plt.show()